# 简介

本项目以CUB 200 2011数据集为样例（以下简称CUB），实现了大作业的简单框架，并就大作业的要点进行讨论。对大作业的要点的探讨，仅作抛砖引玉，欢迎同学们查阅相关资料深入学习，并在课上课下提出讨论。

基于个人理解，本项目代码有诸多已提出但未实现的点，以及提出的可供探讨的问题，可能可以给同学们带来启发。当你对大作业感到迷茫的时候，在本文件中搜索**TODO**定位这些点，可以试着去思考或者实现，有助于回顾或扩展相关知识，以及提升代码理解和能力(注意并不是所有的**TODO**都直接关系到大作业的完成情况，仅供思考)。

关于项目代码有其他任何问题，请对分易联系助教@高向能。

# 目录

0.项目组成和使用

1.模型训练技巧和常见问题

2.迁移学习

3.生成模型

4.CNNs和ViTs 

5.VLM 

6.可解释性

7.鲁棒性（未实现）

8.轻量化模型

9.实验设计：消融、多数据集和参数敏感性实验


# 0.项目组成和使用

本项目文件由以下结构组成

|-- DL2024_proj/

    |-- analysis/ 模型结果输出的文件夹，以及数据统计工具类
    
        |-- result/ 最终结果输出文件夹
        
            |-- xxx/ 按版本号输出的文件夹
            
                |-- foldk.csv 第k折交叉验证的真实标签'true'，预测标签'pred'，预测概率'prob'，图像路径'path'
                
                |-- foldk_report.csv 第k折交叉验证的分类报告
                
                |-- foldk_fc_weight.npy 第k折交叉验证的全连接层权重
        
        |-- mid_feature/ 钩子（hook）获取的中间结果（某层的输入输出特征）输出文件夹
        
            |-- xxx/ 按版本号输出的文件夹
            
                |-- foldk/ 按第k折交叉验证输出的文件夹
                
                    |-- xxx 逐个输出测试集样本的中间结果
                    
        |-- analysis_tools.py 绘出混淆矩阵、roc曲线、计算CAM、保存热力图的类
        
        |-- statistic_result.py 统计评价指标和auc的类
                
    |-- ckpt/ 模型参数输出的文件夹
    
        |-- xxx/ 按版本号输出的文件夹
        
            |-- foldk/ 按第k折交叉验证输出的文件夹
            
                |-- epoch=xxx.pth 模型参数输出，包含迭代次数'epoch'，保存路径'save_dir'，模型参数字典'state_dict'，优化器参数字典'optimizer'
        
    |-- converter/ 文件读取的工具类的文件夹
            
    |-- csv_file/ csv格式的数据集目录的文件夹
                
    |-- data_utils/ 数据读取的工具类的文件夹
    
        |-- data_loader.py 数据读取
    
        |-- transform.py 数据转换和数据增强
            
    |-- datasets/ 数据集的文件夹
    
        |-- CUB_200_2011/ CUB 200 2011数据集
        
            |-- attributes.txt 
            
            |-- CUB_200_2011/ 
            
                |-- attributes 
                
                |-- images 图像数据
                
                |-- parts 
                
                |-- bounding_boxes.txt 
                
                |-- classes.txt.txt 
                
                |-- image_class_labels.txt 
                
                |-- images.txt 目录文件
                
                |-- README
                
                |-- train_test_split.txt 训练集和测试集的划分文件

    |-- gen_dataset/ 生成模型生成的数据集的文件夹
                    
    |-- log/ 模型运行的过程数据输出的文件夹
    
        |-- xxx/ 按版本号输出的文件夹
        
            |-- foldk/ 按第k折交叉验证输出的文件夹
            
                |-- data/ 过程数据输出文件夹
                
                |-- events.xxxxx 命令行输出结果
        
    |-- model/ 模型文件夹
                
    |-- config.py 模型配置
    
    |-- LICENSE 权利声明
    
    |-- main_GAM.py 简单的GAN模型的入口
    
    |-- main_interpretability.py 简单的利用CAM保存热力图的入口
    
    |-- main_VLM.py 简单的VLM模型的入口
    
    |-- main.py 项目模型入口
    
    |-- Readme.ipynb 项目的说明
    
    |-- requirements.txt 依赖库
    
    |-- make_csv.py 制作csv格式的数据集目录的函数
    
    |-- trainer.py 训练器，训练过程主要代码
    
    |-- utils.py 工具类
    
# 数据预处理

1.到数据集官网上阅读数据集相关信息，并将数据数据集下载到datasets文件夹中，至少**包含图像数据，索引文件，训练集和测试集的划分文件**。

2.考虑到不同数据集的结构不同，本项目使用统一建立的csv文件作为索引管理各个数据集，这类文件结构如下：第一列“id”标识数据的路径（项目根目录的相对路径，或者绝对路径），第二列“label”标识数据的实际类别，训练集和测试集分别用不同的csv文件作为索引。

3.修改make_csv.py，为不同数据集编写函数，利用索引文件和划分文件，生成对应数据集的csv索引文件和训练集、测试集的csv索引文件到目录.csv_flie/中（已为CUB编写代码并生成csv索引）。

# 项目代码运行

1.修改config.py，指定网络名称NET_NAME，版本VERSION，**版本决定数据保存的文件夹名，相同版本号会覆盖保存的文件**，以及超参数INIT_TRAINER和SETUP_TRAINER。其中INIT_TRAINER前6个常用参数可以在bash运行时覆盖（详见下文）,具体参数释义如下：
~~~python
INIT_TRAINER = {
    'net_name':NET_NAME, #网络名称，由NET_NAME决定
    'lr':1e-3,  #初始学习率
    'n_epoch':2, #迭代次数
    'num_classes':200, #分类的类别别数
    'image_size':256, #图像的边长
    'batch_size':100, #批次大小
    'train_mean':CUB_TRAIN_MEAN, #训练集的均值
    'train_std':CUB_TRAIN_STD, #训练集的标准差
    'num_workers':2, #dataloader的线程数
    'device':DEVICE, #运行使用的设备
    'pre_trained':PRE_TRAINED, #是否载入模型预训练权重，由PRE_TRAINED决定
    'weight_path':WEIGHT_PATH, #模型预训练权重的路径，由WEIGHT_PATH决定
    'weight_decay': 1e-4, #优化器L2正则化的λ参数
    'momentum': 0.9, #SGD优化器的参数
    'gamma': 0.1, #MultiStepLR学习率策略的参数
    'milestones': [30,60,90], #MultiStepLR学习率策略的参数
    'T_max':5, #CosineAnnealingLR学习率策略的参数
    'use_fp16':True #use_fp16是否使用半精度训练
 }

SETUP_TRAINER = {
    'output_dir':'./ckpt/{}'.format(VERSION), #模型权重输出路径
    'log_dir':'./log/{}'.format(VERSION), #模型过程输出路径
    'optimizer':'AdamW', #优化器
    'loss_fun':'Cross_Entropy', #损失函数选择
    'class_weight':None, #损失函数的类权重
    'lr_scheduler':'MultiStepLR' #学习率策略选择
}
~~~

2.修改main.py，指定所选数据集的csv索引文件路径，如下：

~~~python
if 'train' in args.mode:
    csv_path = './csv_file/cub_200_2011.csv_train.csv'
...
        
elif 'inf' in args.mode:
    test_csv_path = './csv_file/cub_200_2011.csv_test.csv'

~~~

3.开始训练和推理，进行训练时，config.py中PRE_TRAINED参数设置为False时，将从头开始训练，并**清空覆盖当前版本的输出文件夹**；PRE_TRAINED参数设置为True时，将载入当前版本上次训练权重，并从上次训练的终点继续训练，**此时参数'n_epoch'应该大于上次训练的设置值**。

进行推理时，config.py中PRE_TRAINED参数设置为False时，将（没有训练）从头开始推理；PRE_TRAINED参数设置为True时，将载入当前版本上次训练权重，并从上次训练的终点开始推理，推理任何时候只执行一个迭代。

在项目根目录下，

单折、多折训练
~~~bash
python main.py -m train
python main.py -m train-cross
~~~

单折、多折推理
~~~bash
python main.py -m inf
python main.py -m inf-cross
~~~

单折、多折推理（不需要结果输出）
~~~bash
python main.py -m inf -s n
python main.py -m inf-cross -s n
~~~

config.py中的INIT_TRAINER前6个常用参数可以在bash运行时覆盖，例如：
~~~bash
python main.py -m train -n resnet34 -l 1e-4 -e 1000 -c 200 -is 224 -bs 1000
~~~
表示单折训练，网络模型选择resnet34，初始学习率为1e-4，迭代次数1000，类别数200，图像大小224*224，批次大小1000，**这6个参数都是可选参数，在不输入这些参数时，采用config.py中的INIT_TRAINER对应参数的值**。

# 模型输出以及查看和修改

模型输出主要分为  
推理过程：最终结果和中间结果输出  
训练过程：模型参数和过程数据输出  

***最终结果输出:***.analysis/result/xxx/foldk/foldk.csv，为第k折交叉验证的真实标签'true'，预测标签'pred'，预测概率'prob'，图像路径'path'；.analysis/result/xxx/foldk_report.csv，为第k折交叉验证的分类报告;.analysis/result/xxx/foldk_fc_weight.npy，为了方便做CAM输出的全连接层的权重。以上csv格式文件使用pandas读取或者直接打开查看；npy格式文件使用numpy读取。

在main.py中搜索'csv_file'，在trainer.py中搜索'result'，可以查看和修改最终结果输出相关代码。

***中间结果输出:***.mid_feature/xxx/foldk/xxx，为逐个输出测试集样本的钩子获取的中间结果，保存为hdf5格式的字典，读取方式为使用.converter/common_util.py的函数:

~~~python
def hdf5_reader(data_path, key=None)
~~~


其中，key=['feature_in', 'feature_out']，分别代表钩子获取的某层的输入和输出特征，。利用可视化方法可以对中间结果进行可视化（见6.可解释性）。

在main.py中搜索'save_as_hdf5'，在trainer.py中搜索'feature_in'，可以查看和修改中间结果输出相关代码。

***模型参数输出:***.ckpt/xxx/foldk/epoch=xxx.pth，格式为torch.save直接保存的字典（用于加载模型参数），包含key-value对：迭代次数'epoch'，保存路径'save_dir'，模型参数字典'state_dict'，优化器参数字典'optimizer'。读取方式为

~~~python
data = torch.load(weight_path)['key']
~~~

在trainer.py中搜索'saver'，可以查看和修改模型参数输出相关代码。

***过程数据输出:***.log/xxx/foldk/，首先使用tensorboard指定日志目录为log/:

~~~bash
tensorboard --logdir log
~~~

然后访问<http://localhost:6006/>，如果是命令行服务器则需先将6006端口代理到本地，然后本地访问上述网址。

在trainer.py中搜索'writer.add_scalar'，可以查看和修改过程数据输出相关代码。

# 主要代码的函数释义

# main.py

~~~python
def get_cross_validation(path_list, fold_num, current_fold): #生成多折交叉验证数据集
    
def get_parameter_number(net): #计算网络参数
~~~

# trainer.py

~~~python  
class VolumeClassifier(object):
#分类器类
    
    def trainer(self,
                train_path,
                val_path,
                label_dict,
                output_dir=None,
                log_dir=None,
                optimizer='Adam',
                loss_fun='Cross_Entropy',
                class_weight=None,
                lr_scheduler=None,
                cur_fold=0): 
        #定义和初始化训练器，训练过程（训练和验证）外层循环逻辑
    
    def _train_on_epoch(self, epoch, net, criterion, optimizer, train_loader, scaler):
        #每个训练迭代的逻辑
    
    def _val_on_epoch(self, epoch, net, criterion, val_path, label_dict):
        #每个验证迭代的逻辑

    def hook_fn_forward(self, module, input, output):
        #钩子定义
    
    def inference(self,
                  test_path,
                  label_dict,
                  net=None,
                  hook_fn_forward=False):
        #推理逻辑
    
    def _get_net(self, net_name):
        #根据模型名称，实例化对应模型
    
    def _get_loss(self, loss_fun, class_weight=None):
        #根据损失函数名称，实例化损失函数

    def _get_pre_trained(self, weight_path):
        #根据模型预训练权重的路径，为模型载入预训练权重
        
class AverageMeter(object):
    #在模型训练、验证和推理过程中记录当前值、综合、计数和均值的类

def accuracy(output, target, topk=(1, )):
    #计算Topk准确率
    
class EarlyStopping(object):
    #早停机制类
~~~

# 1.模型训练技巧和常见问题

# 随机种子

随机种子可以在很大程度上影响模型的结果<https://arxiv.org/abs/2109.08203>，但我们的模型优化并不是去寻找一个好的随机种子，针对这点一般有两种处理方案：

1.使用固定的无意义的随机种子，比如0。本项目代码采用方案，可以通过在训练过程前进行如下设置实现：
~~~python 
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed_all(0)
~~~

2.**TODO**使用随机的随机种子，重复多次实验，在统计学意义上记录结果，更科学但更费时的方案。

# 参数和超参数

参数（parameter）是模型内部的配置变量，可以用数据去估计参数的值（即训练模型）。参数通常定义了模型的功能和行为，例如人工神经网络中的权重，支持向量机中的支持向量，线性回归或逻辑回归中的系数。

超参数（hyper parameter）是模型外部的配置变量，不能用数据去估计超参数的值（通常直接设置，或者使用启发式方法）。超参数通常定义了估计模型参数的过程，例如学习率、batch size、迭代代数，支持向量机的C和sigma等。config.py中定义的都是超参数。

# 训练、验证和推理

规范上而言，数据集首先划分为**独立的**训练集和测试集，**测试集模拟的是模型真实投入使用场景后，面对的未知数据，因此除了最后的推理过程，模型应该保持测试集的不可见（未知性）**。因此在数据预处理时，应该使用训练集的值对测试集进行标准化。具有相同的数据集划分是模型结果得以比较的前提。

训练过程中，再次将训练集的一部分划分为验证集（例如8:2或7:3的比例），使用剩下的训练集来训练模型（估计参数的值），使用验证集（**而不是测试集**），来评估训练的成果并人工调整模型的超参数，减轻过拟合和欠拟合现象等。

推理过程中，直接讲测试集输入训练好的模型（提前调整好超参数），使用输出结果作为模型的最终性能评估。**面向测试集进行调整参数和超参数是不规范的行为**，如果测试集结果不理想，可以重新进行训练过程再进行推理过程（严格意义上推理过程只能执行一遍，想一想实际的应用场景）。

注意到验证集是训练集的一部分，但是不参与模型的训练过程。为了最大程度的使用到整个训练集，**TODO**训练完毕后，可以以相同的参数，使用整个训练集进行训练，再进行推理。

# GPU资源

机器学习模型通常在GPU上加速训练，GPU资源主要指GPU的算力和显存，算力决定运行时间，显存决定是否可以运行。越复杂的模型，越大的数据占用越多的算力和显存，当算力和显存不足时，代码将运行变慢或无法运行（显存不足）。

如何优化GPU资源利用：

1.优化模型代码和训练代码，任重道远。

**TODO**对于较小的模型（ResNet18、ResNet34等），参考模型代码、模型结构图，估计模型的参数量以及占用空间，对照函数输出的模型参数量和查询到的模型参数量，看看估计是否准确，相差的参数可能在哪里？
~~~python
def get_parameter_number(net)
~~~

2.使用混合精度（单精度和半精度混合）训练，通常而言，tensor以单精度浮点型存储数据（float32），大小为32Bits；半精度浮点型（float16）大小为16Bit。使用半精度可以降低一半的显存占用，以及提升计算速度。在config.py中将'use_fp16'设置为True使用混合精度训练（本项目代码默认设置），具体执行代码在trainer.py中搜索'use_fp16'（使用torch.cuda.amp.autocast实现torch.cuda.amp.GradScaler ）。

**TODO**思考为什么不全部使用半精度，单精度和半精度的范围、精度分别是什么，混合精度训练中哪些部分使用了半精度，autocast和GradScaler的实现原理？

3.降低batch size，使得一个batch输入更少的数据，代价是更多的计算时间，以及（很可能的）模型性能下降。

**TODO**思考为什么会引起模型性能下降，在资源充足的情况下，batch size是不是越大越好？联系优化器的优化过程。

4.冻结部分模型参数，在某些情形下（特别是迁移学习），**TODO**可以通过冻结模型的部分参数，使其不参与训练，来降低GPU资源占用，思考这样做的利弊。

5.使用多GPU并行计算。

# 多GPU并行

在有多个GPU资源的前提下，可以进行并行计算，分为数据并行和模型并行，多GPU并行不会影响模型结果。

数据并行：将一个batch的数据分配给多个GPU，分别计算并汇总到某一个主GPU上，比较常用且简单的并行方法，速度瓶颈是汇总的GPU；通过修改config.py中的DEVICE参数实现，例如DEVICE = '0','1'表示代码将在0和1号GPU上进行数据并行（前提是这些GPU可用）。

**TODO**代码并行：将模型代码的不同部分拆分成流水线，分配不同的GPU进行计算，需要对模型代码进行流水线上的重构设计，速度瓶颈是流水线中耗时最长的部分。

混合并行：以上两种方式混合使用。

# 交叉验证

交叉验证就是进行多次训练集和验证集划分，在不同的训练集和验证集上进行训练和验证，综合各个验证结果对模型进行评估和调整超参数。交叉验证可以降低评估结果的偶然性，提高模型评估的准确性，代价是成倍的训练时长。通常使用的交叉验证方法有k折交叉验证、留一法交叉验证等。main.py实现了k折交叉验证，通过修改config.py中的FOLD_NUM参数修改交叉验证的折数。

**TODO**为什么不使用留一法交叉验证，原理和利弊是什么，适用于什么情形？

除此以外，对于很小的数据集，不便于划分**独立的**训练集和测试集时，可以用交叉验证的统计学结果，来一定程度上代替独立测试集的结果。

# 损失函数

损失函数（loss function）用来评估模型输出值和真实值之间的损失（loss），loss通过反向传播用于模型梯度的更新。本项目代码默认使用交叉熵损失（Cross Entropy）。在trainer.py中搜索_get_loss进行详细设置。

不同的损失函数适用于不同问题，例如，类别不平衡问题通常具有特别设计的损失函数。**TODO**针对你面临的问题特点，尝试和设计不同的损失函数，对比模型性能。

# 优化器

在loss通过反向传播对模型梯度进行更新后，优化器（optimizer）负责根据模型梯度进行模型权重更新，在trainer.py中搜索_get_optimizer进行详细设置。

不同的优化器适用于不同问题和模型，**TODO**针对你面临的问题和模型特点，尝试和设计不同的优化器，对比模型性能。

# 学习率

学习率（learn rate, lr）是优化器的参数，决定模型权重更新的步长，通常情况下，学习率初始较大且随着迭代减小。学习率越小，达到同样的训练效果需要的迭代次数越多。通过修改config.py中的lr参数修改初始学习率。

# 学习率下降策略

学习率下降策略（lr scheduler）决定学习率如何下降，在trainer.py中搜索_get_lr_scheduler进行详细了解和设置。

不同的学习率下降策略适用于不同问题和模型，**TODO**针对你面临的问题和模型特点，尝试和设计不同的学习率下降策略，对比模型性能。

# 评价指标

评价指标（metrics）用于评价模型的性能表现，对于分类问题，准确率（Accuracy）是最普遍使用的评价指标。即使是分类问题中，准确率不是万能的，不同的评价指标适用于不同问题以及评价模型的不同方面的性能，例如，精确度（precision），召回率（recall），F1值（F-Score），AUROC，混淆矩阵等也是分类问题常用的评价指标。

**TODO**针对你面临的问题特点和倾向的模型性能，了解和尝试不同的评价指标。

# 过拟合和欠拟合

过拟合和欠拟合指的是模型对于训练集的拟合情况，即训练过头和训练不够。过拟合表现为模型在训练集性能很好但是在验证集上性能很差，欠拟合表现为模型在训练集和验证集上性能都很差。我们优化模型的目标是训练集和验证集的性能都很好（通常此时训练集的性能会好过验证集性能）。

处理过拟合的方法主要有早停机制、Lp正则、Dropout、批标准化、数据增强、集成学习等。

处理欠拟合的方法主要有数据归一化，增加迭代次数（结合使用较小的lr），增大batch size，换用更复杂的模型（例如更深的网络层数）等。

# 早停机制

早停机制（early stopping）指的是在训练过程中，隔x次迭代验证集效果无提升时，提前终止训练过程防止过拟合，同时节省资源和时间。在trainer.py中搜索EarlyStopping进行详细了解和设置。

# Lp正则

Lp正则（Lp regularization）指的是给损失函数加入Lp范数（Lp norm）形式的惩罚因子。Lp范数用来计算向量投影和表示距离，常用的有L1范数、L2范数和L∞范数。

torch.optim集成的优化器自带L2正则，在定义优化器的时候设置'weight_decay'参数可以设置L2正则的λ参数，例如（本项目代码中可以直接修改config.py的超参数INIT_TRAINER中'weight_decay'）：

~~~python
optimizer = torch.optim.Adam(net.parameters(),lr=lr,weight_decay=self.weight_decay)
~~~

自定义正则**TODO**：在trainer.py中搜索_get_loss，修改损失函数的相关代码，加入L1正则或可自定义的Lp正则。

# Dropout

Dropout是神经网络中一种特殊的层，一般在全连接层后插入。是一种通过在每个训练批次的前向传播中，随机丢弃一部分神经元（同时丢弃其对应的连接边，即这些神经元不工作），来减少过拟合的正则化方法。

vision_transformer.py已经实现Dropout正则化，通过设置config.py的超参数INIT_TRAINER中'dropout'可以设置dropout的比例。**TODO**vision_transformer.py中还提供一个参数'attention_dropout'，查看代码（和论文），分析两种dropout的作用区别，并在trainer.py和config
.py中搜索dropout，修改相关代码，添加'attention_dropout'设置。

resnet.py中没有实现dropout，**TODO**阅读论文理解这是为什么？<https://arxiv.org/abs/1603.05027>

# 标准化

批标准化（Batch Normalization）是神经网络中一种特殊的层，一般在激活层前插入，主要用于加速网络收敛，也有降低过拟合的效果。通过调用torch.nn.BatchNorm2d实现。**TODO**批标准化出现以后，dropout已经较少被主流网络使用，查询论文并思考这是为什么？

**TODO**查看torch.nn的api:  <https://pytorch.org/docs/stable/nn.html#normalization-layers>，在resnet.py中修改代码，尝试并思考批标准化替换为其他三种标准化的效果(
层标准化（Layer Normalizaiton）,实例标准化（Instance Normalization）,组标准化（Group Normalization）)。

# 数据增强

数据增强（Data Augmentation），指的是对原数据施加旋转、反转、高斯噪音等随机小扰动，或者通过超采样、负采样等技术。使输入模型的数据在原数据集基础上更具多样化，一定程度上扩充了数据集，减少对保质保量的训练数据的依赖，并提升模型的训练效果。

通常数据集需要多种数据增强方法联合使用，通过torchvision.transforms.Compose实现。  
在train.py中搜索transforms.Compose可以设置数据集使用的数据增强方法及其顺序，在transform.py中进行数据增强的详细了解和设置。  
**TODO**修改和设计新的数据增强方法。

# 集成学习

集成学习（ensemble learning）是指将多个训练好的基学习器（base learners）通过多种方式结合进行推理的模型，集成学习有效的理论前提是基学习器好且好得不同：
1.基学习器的推理优于随机推理；
2.基学习器的错误互相独立。
**TODO**尝试理解这是为什么？

由于资源的成倍开销，集成学习的基学习器通常用较为轻量化的模型，例如决策树、SVM等，在各个机器学习、数据挖掘竞赛中集成学习被用于追求极致地提高模型性能（刷点数）。近年来深度学习特别是transformer大模型范式的出现，集成学习热度有所降低（复杂模型的集成学习资源开销过大）。

# 权重初始化

ResNet使用He Kaiming的权重初始化方法torcm.nn.init.kaiming_normal_()，查看torch.nn的api: <https://pytorch.org/docs/stable/nn.init.html>，理解具体操作。

# 2.迁移学习

迁移学习指的是首先将模型在额外数据上进行预训练，再将预训练模型“迁移”到现有问题的技术。
迁移学习技术有助于提升模型在小数据集上的表现，使用他人发布的预训练模型还可以减少训练的时间和资源消耗，在以transformer为代表的大模型领域被广泛应用。

根据如何将预训练模型“迁移”到现有问题上，以下是几个迁移学习的思想：

Transfer Learning：冻结预训练模型的全部卷积层（主要是提取特征的层，消耗资源较多），只训练自己定制的全连接层（主要是分类器层，消耗资源较少）。

Extract Feature Vector：先计算出预训练模型的卷积层对所有训练和测试数据的特征向量，然后抛开预训练模型，只训练自己定制的简配版全连接网络（行为上和上一条类似）。 

Fine-tune：冻结预训练模型的部分层（通常是靠近输入的多数卷积层），训练剩下的层（通常是靠近输出的部分卷积层和全连接层）（**TODO**为什么是这样）。

目前Fine-tune是最常用的迁移学习技术，称为微调，如何对预训练模型进行微调，是一个值得深入研究的话题。本项目代码的resnet.py和vision_transformer.py原样拷贝了torchvision库的相关内容，**TODO**尝试修改resnet.py和vision_transformer.py，实现下载并使用预训练模型的权重，并设计微调策略，冻结部分层训练其他层。

PEFT库<https://github.com/huggingface/peft>是一个很好的微调库，包含许多SOTA方法，**TODO**尝试了解、学习并调用、移植来提高你的模型性能。


# 3.生成模型

通过生成模型来生成额外数据，广义上属于超采样的数据增强技术，最经典的生成模型为生成对抗网络（Generative Adversarial Network, GAN）和扩散模型（Diffusion Models），**TODO**查询并深入了解这二者代表的思想的异同。

本项目代码实现了简单的DCGAN，由于生成模型的训练逻辑的不同，编写了另外的入口函数（main_gan.py）。该入口函数较为粗糙，**TODO**尝试对照trainer.py、main.py、config.py、规范代码并补全相关功能。

注意到，DCGAN不涉及图像的标签，**TODO**如何获得生成的图像的标签也是一个值得探讨的话题，查阅并尝试其他生成模型，它们是如何解决这个问题。

# 4.CNNs和ViTs

卷积神经网络（Convolutional Neural Networks, CNNs）多年来一直是计算机视觉领域的重要甚至主要技术。2017年Google提出的Transformers最开始应用于自然语言处理领域。通过一些细小的设计（什么设计？），可以使得自注意力模型对图像进行处理，这样的结构称为视觉Transformers，ViTs。

**TODO**本项目代码的resnet.py和vision_transformer.py，查询、总结并思考二者的异同，例如，CNNs和ViTs的基本操作，卷积和自注意力的计算过程以及相关性质，局部性和全局性的理解；CNNs和ViTs分别的适用问题、数据集；迁移学习在CNNs和ViTs的应用以及区别等。对二者的性质具备一定程度的了解，有助于根据不同问题选择针对的模型。

# 5.VLM

为了减轻训练对人工标注数据数据的依赖，以及研究如何将视觉模型零样本（zero-shot）迁移到其他问题上。研究者提出视觉语言模型（Vision-Language Models）通过从互联网上大量的图像-文本对中学习到丰富的视觉-语言关联，能够在只使用单一VLM模型的情况下实现对各种视觉任务的zero-shot迁移。

目前主流的以CLIP为典型代表的Vision-Language Model(VLM)预训练方法可以大致分为3个关键模块：

文本特征提取模块，通常采用Transformers结构及其一系列变体作为基础结构。

图像特征提取模块，通常采用CNNs或者Transformers（如ViT）来提取图像特征。

特征融合模块，在VLM预训练模型中，最关键的问题是将文本和图像这两种模态的信息建立联系。

特征对齐模块中，以目标函数进行分类的话，大致可以分为三类目标函数：

对比式（Contrastive Objectives），生成式（Generative Objectives），对齐式（Alignment Objectives）。

**TODO**根据一个简单的调用Clip模型的示例代码main_VLM.py，学习并实现VLM模型，使得你的模型能够利用额外数据集进行训练，并零样本迁移到其他问题上。

# 6.可解释性
目前在计算机视觉研究中，研究者通常将模型部署为一个黑盒，注重于优化准确度等性能，如何对模型的作用原理提供一个令人满意的解释是一个研究热点。

模型可解释性的通常由类激活映射（Class Activation Mapping, CAM）及其相关方法实现，类激活图可以显示模型权重如何聚焦和转移。

本项目代码main_interpretability.py实现了简单的CAM方法保存热力图的代码，实现过程（以ResNet为例）：

1.使用钩子（hook）获取模型的中间结果，这里研究整个模型，因此选择最后一层卷积层的输出特征，默认的钩子绑定的是池化层的输入和输出特征，池化层的输入特征即为最后一层卷积层的输出特征；  
2.利用特征和全连接层计算各个类的CAMs（analysis.analysis_tools.calculate_CAMs）;  
3.将预测类的CAMS叠加到原图像上获得基于CAM的热力图（analysis.analysis_tools.save_heatmap）。

**TODO**
在本项目代码的main_interpretability.py中，只实现了生成单张图片的热力图，编写批量生成的代码。

**TODO**
在本项目代码的trainer.py中，搜索hook_fn_forward对钩子进行进行了解和设置，在测试过程中通过hook获取其他层（起主要特征提取作用的层）的输出特征并获得基于CAM的热力图。

**TODO**
尝试基于其他CAM变种方法进行可解释性研究，本项目代码实现的CAM获取卷积层的输出特征，不适用于ViTs，查阅并尝试对ViTs进行可解释性研究。

# 7.鲁棒性

鲁棒性指的是模型对抗错误的能力，通常使用专门设计的对抗样本来研究模型鲁棒性，例如FGSM（Fast Gradient Sign Attack）通过在输入数据上叠加梯度形成扰动，来对模型进行攻击；PGD（Projected Gradient Descent）可以看作是FGSM的多步迭代，通过叠加多步梯度形成扰动。本项目没有实现该部分代码，试着自己上手实现一下。

**TODO**参考torch关于FGSM的api: <https://pytorch.org/tutorials/beginner/fgsm_tutorial.html>;  
PGD论文<https://arxiv.org/abs/1706.06083>;  
PGD代码<https://github.com/MadryLab/mnist_challenge>;  
trainer.py的推理过程;  
编写一个PGD的推理过程（也可以单独编写入口函数），实现模型鲁棒性的研究，更进一步的，可以尝试其他攻击方法。

**TODO**思考如何提升模型的鲁棒性，例如可不可以使用对抗样本作为训练，起到什么效果？

# 8.轻量化模型

压缩模型参数，设计轻量化模型，主要有以下方法：

1.模型剪枝：指在预训练模型的基础上删除冗余参数，例如删除某些神经元的连接、某些层的通道、某些层等。可能会带来性能的下降，需要对模型反复微调来寻求性能和参数的均衡。

2.参数量化：指用较低精度（半精度、8位整型甚至更低）来存储模型参数，不同于torch自动的混合精度，参数量化需要根据不同参数对精度的需求，对模型进行重新设计。

3.低秩矩阵分解：通过线性代数的低秩矩阵分解(low-rank matrix factorization)，实现卷积核矩阵的稀疏化，从而减少空间占用。

4.参数共享：与模型剪枝类似，利用存在冗余参数的特点，设计映射将多数参数映射到少数数据上。

5.设计紧凑模型结构：通过构造特殊的卷积、层和结构等方式巧妙地减少参数数量，例如SqueezeNet<https://arxiv.org/abs/1602.07360>，  
MobileNet<https://arxiv.org/abs/1704.04861>，  
ShuffleNet<https://arxiv.org/abs/1707.01083>等。

6.知识蒸馏：预先训练好的教师模型，具有完善的结构、参数和很好的性能；学生模型结构紧凑。将教师模型的输出作为（soft target）与学生模型的输出（hard target）一起计算loss，从而将教师模型的“知识”迁移到学生模型中，使学生模型达到与教师模型相当的性能，实现模型的压缩。


**TODO**查询并尝试一个或几个方法，压缩你的模型参数，设计一个轻量化模型。

# 9.实验设计：消融、多数据集和参数敏感性实验

通常而言，一个完善的科技论文实验部分包括：对比实验、可解释性研究、消融实验、多数据集实验、参数敏感性实验等。

**TODO**在你的报告、论文中去尽可能完善地实现它们：

对比实验：选取经典方法和SOTA（state of the art）方法作为baseline对比模型性能。

可解释性研究：对模型主要特征提取部分（创新部分）或模型整体的输出特征进行可视化，并通过模型行为解释模型性能。

消融实验：通过去除模型中的某个（某些）部分，对比模型性能的下降情况，研究模型主要是哪个（哪些）部分在起作用。

多数据集实验：选用多个数据集进行实验对比，可以包含完整的训练过程，也可以少样本、零样本地将模型迁移到新的数据集上，研究模型的泛化能力。

参数敏感性实验：选取有特点的、感兴趣的、起到关键作用的超参数进行研究，改变（通常是网格搜索）超参数的值进行模型性能的对比，进而分析模型行为，解释模型性能。
